In [14]:
import numpy as np
import tensorflow as tf
import keras
from tensorflow.keras import layers,Model
from tensorflow.keras import datasets
from tensorflow.keras.losses import MeanSquaredError, KLDivergence, binary_crossentropy
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
import matplotlib.pyplot as plt
from numpy import cov, trace, iscomplexobj,asarray
from numpy.random import randint
from scipy.linalg import sqrtm
from skimage.transform import resize

In [3]:
(x_train, y_train), (x_test, y_test) = datasets.cifar10.load_data()

170498071/170498071 ━━━━━━━━━━━━━━━━━━━━ 8s 0us/step


In [4]:
x_train = x_train.astype('float32')/255.0
x_test = x_test.astype('float32')/255.0

In [5]:
latent_dim = 1024
image_shape = x_train[0].shape
mse_loss = MeanSquaredError()
kl_loss = KLDivergence()

In [8]:
class VAE(keras.Model):
    def __init__(self,latent_dim,image_shape,beta):
        super(VAE,self).__init__()

        self.latent_dim=latent_dim
        self.image_shape=image_shape
        self.beta=beta

        self.encoder=tf.keras.Sequential([
            layers.Input(shape=image_shape,name="Encoder_Input_layer"),

            layers.Conv2D(32,3,strides=2,activation="relu",padding='same',name="Encoder_Conv2D_2"),
            layers.Conv2D(64,3,strides=2,activation="relu",padding='same',name="Encoder_Conv2D_4"),
            layers.Conv2D(128,3,strides=2,activation="relu",padding='same',name="Encoder_Conv2D_6"),
            layers.Flatten(name="Encoder_Flatten"),
        ])

        self.z_mean=layers.Dense(latent_dim,name="z_mean")
        self.z_log_var=layers.Dense(latent_dim,name="z_log_var")

        self.decoder=tf.keras.Sequential([
            layers.InputLayer(input_shape=(latent_dim,)),
            layers.Dense(8*8*64,activation='relu'),
            layers.Reshape((8,8,64)),
            layers.Conv2DTranspose(64,kernel_size=3,strides=2,padding='same',activation='relu'),
            layers.Conv2DTranspose(32,kernel_size=3,strides=2,padding='same',activation='relu'),
            layers.Conv2DTranspose(3,kernel_size=3,strides=1,padding='same',activation='sigmoid'),
       ])

        self.total_loss_tracker=keras.metrics.Mean(name="total_loss")
        self.reconstruction_loss_tracker=keras.metrics.Mean(name="reconstruction_loss")
        self.kl_loss_tracker=keras.metrics.Mean(name="kl_loss")

        #build the encoder
    def encode(self,data):
        x=self.encoder(data)
        z_mean,z_log_var=self.z_mean(x),self.z_log_var(x)
        return z_mean,z_log_var

    def reparameterize(self,z_mean,z_log_var):
        batch=tf.shape(z_mean)[0]
        dim=tf.shape(z_mean)[1]
        epsilon=tf.keras.backend.random_normal(shape=(batch,dim))
        z=z_mean+tf.exp(0.5*z_log_var)*epsilon
        return z

    def call(self,x):
        mean,logvar=self.encode(x)
        z=self.reparameterize(mean,logvar)
        x_recon=self.decoder(z)
        return x_recon,mean,logvar

    def decode(self,data):
        return self.decoder(data) # Pass data to the decoder

    @property
    def metrics(self):
        return [
            self.total_loss_tracker,
            self.reconstruction_loss_tracker,
            self.kl_loss_tracker
        ]

    def recon_loss(self,data,reconstruction):
        return tf.reduce_mean(binary_crossentropy(data,reconstruction))

    def kl_divergence(self,z_logvar,Z_mu):
        kl=-0.5*tf.reduce_mean(1+z_logvar - Z_mu**2- tf.math.exp(z_logvar)) # Corrected variable name and added closing parenthesis
        return self.beta*kl

    def train_step(self,data):
        with tf.GradientTape() as tape:
            z_mean,z_log_var=self.encode(data)
            z=self.reparameterize(z_mean,z_log_var)
            reconstruction=self.decode(z)

            reconstruction_loss=self.recon_loss(data,reconstruction)
            kl_loss=self.kl_divergence(z_log_var,z_mean)

            total_loss=reconstruction_loss + kl_loss

        grads=tape.gradient(total_loss,self.trainable_weights)
        self.optimizer.apply_gradients(zip(grads,self.trainable_weights))
        self.total_loss_tracker.update_state(total_loss)
        self.reconstruction_loss_tracker.update_state(reconstruction_loss)
        self.kl_loss_tracker.update_state(kl_loss)
        return {
            "loss":self.total_loss_tracker.result(),
            "reconstruction_loss":self.reconstruction_loss_tracker.result(),
            "kl_loss":self.kl_loss_tracker.result()
        }

In [15]:
vae=VAE(latent_dim,image_shape,0.3)

In [16]:
vae.compile(optimizer=keras.optimizers.Adam())
early_stopping=EarlyStopping(monitor='loss',patience=10,verbose=1)
history=vae.fit(x_train,epochs=50,batch_size=512,callbacks=[early_stopping])

Epoch 1/50
98/98 ━━━━━━━━━━━━━━━━━━━━ 18s 77ms/step - kl_loss: 5.8864e-04 - loss: 0.6877 - reconstruction_loss: 0.6872
Epoch 2/50
98/98 ━━━━━━━━━━━━━━━━━━━━ 9s 33ms/step - kl_loss: 0.0080 - loss: 0.6276 - reconstruction_loss: 0.6196
Epoch 3/50
98/98 ━━━━━━━━━━━━━━━━━━━━ 5s 33ms/step - kl_loss: 0.0108 - loss: 0.6190 - reconstruction_loss: 0.6082
Epoch 4/50
98/98 ━━━━━━━━━━━━━━━━━━━━ 5s 34ms/step - kl_loss: 0.0125 - loss: 0.6139 - reconstruction_loss: 0.6014
Epoch 5/50
98/98 ━━━━━━━━━━━━━━━━━━━━ 5s 33ms/step - kl_loss: 0.0131 - loss: 0.6120 - reconstruction_loss: 0.5988
Epoch 6/50
98/98 ━━━━━━━━━━━━━━━━━━━━ 5s 33ms/step - kl_loss: 0.0136 - loss: 0.6105 - reconstruction_loss: 0.5969
Epoch 7/50
98/98 ━━━━━━━━━━━━━━━━━━━━ 5s 34ms/step - kl_loss: 0.0138 - loss: 0.6092 - reconstruction_loss: 0.5954
Epoch 8/50
98/98 ━━━━━━━━━━━━━━━━━━━━ 5s 33ms/step - kl_loss: 0.0135 - loss: 0.6072 - reconstruction_loss: 0.5936
Epoch 9/50
98/98 ━━━━━━━━━━━━━━━━━━━━ 5s 34ms/step - kl_loss: 0.0132 - loss: 0.6042

In [17]:
def generate_images_from_latent_vectors(vae, num_images_to_generate):
    random_latent_vectors = np.random.normal(size=(num_images_to_generate, latent_dim))
    generated_images = vae.decoder.predict(random_latent_vectors)
    return generated_images

generated_images = generate_images_from_latent_vectors(vae, 20)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 427ms/step


In [18]:
def plot_real_and_generated_images(real_images,generated_images):
  plt.figure(figsize=(20,10))


  for i in range(num_images_to_generate):
   plt.subplot(2,num_images_to_generate,i+1)
   plt.imshow(real_images[i])
   plt.title('Generated_images')
   plt.axis('off')


  for i in range(num_images_to_generate):
   plt.subplot(2,num_images_to_generate,i+1+num_images_to_generate)
   plt.imshow(generated_images[i+10])
   plt.title('Generated_images')
   plt.axis('off')

  plt.tight_layout()
  plt.show()

#plot real and generated images side by side vertically
plot_real_and_generated_images(x_test,generated_images,5)

TypeError: plot_real_and_generated_images() takes 2 positional arguments but 3 were given